<a href="./a.py"> Download File </a>

In [ ]:
!pip install -q ../input/flex-yolo-pack/torch-1.7.1+cu110-cp37-cp37m-linux_x86_64.whl
!pip install -q ../input/flex-yolo-pack/torchaudio-0.7.2-cp37-cp37m-manylinux1_x86_64.whl
!pip install -q ../input/flex-yolo-pack/torchvision-0.8.2+cu110-cp37-cp37m-linux_x86_64.whl
# !pip install  --user ../input/flex-yolo-pack/pycocotools-2.0.4/pycocotools-2.0.4.tar
# !pip install -q ../input/flex-yolo-pack/coremltools-5.1.0-cp37-none-manylinux1_x86_64.whl
!pip install -q ../input/flex-yolo-pack/addict-2.4.0-py3-none-any.whl
# !pip install -q ../input/flex-yolo-pack/onnx-1.10.2-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
# !pip install -q ../input/flex-yolo-pack/scikit_learn-0.19.2-cp37-cp37m-manylinux1_x86_64.whl
# !pip install -q ../input/flex-yolo-pack/thop-0.0.31.post2005241907-py3-none-any.whl
!pip install -q ../input/timm-model/timm-0.3.2-py3-none-any.whl

In [ ]:
# from IPython.core.display import HTML
# HTML("<script>flexible_yolo_with_attention_mechnism.notebook.kernel.restart()</script>")

In [ ]:
# import numpy as np
# import cv2
# import matplotlib.pyplot as plt
# tmp=np.load('../input/tensorflow-great-barrier-reef/example_test.npy')
# from PIL import Image
# # print(tmp[0].shape)
# # print(tmp[0])

# # im = Image.fromarray(tmp[0])
# # imt=cv2.imread(im)

# # print(tmp.shape)
# # im.show()
# t=cv2.imread('../input/test-img/1_5296-1-0.jpg')
# print(t.shape)
# print(t)
# # plt.imshow(tmp[0], cmap='gray')
# # plt.imshow(t)
# # # print(t.shape)
# # plt.imshow(rgb, cmap='gray')
# # plt.show()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
sys.path.append('../input/flexible-yolov5/flexible-yolov5/scripts')
sys.path.append('../input/flexible-yolov5/flexible-yolov5')
import torch
from PIL import Image
import ast

In [ ]:
data = '''
starfish
'''
with open('./class.txt', 'w') as fp:
    fp.write(data)

In [ ]:

# def predict(model, img):
#     height, width = img.shape[:2]
#     results = model(img)  # custom inference size
#     preds   = results.pandas().xyxy[0]
#     bboxes  = preds[['xmin','ymin','xmax','ymax']].values
#     if len(bboxes):
#         bboxes  = voc2coco(bboxes,height,width).astype(int)
#         confs   = preds.confidence.values
#         return bboxes, confs
#     else:
#         return [],[]
    
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            
            xmin, ymin, w, h=int(xmin), int(ymin), int(w), int(h)
            conf             = confs[idx]
            annot +='{:.2f} {} {} {} {}'.format(conf, xmin, ymin, w, h)

#             annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

def show_img(img, bboxes, bbox_format='yolo'):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((800, 400))

In [ ]:
# img.shape

In [ ]:
from detector import *
from glob import glob
pt_path = '../input/yolo-flex-model/yolo_flex.pt'
imgs = ['../input/test-img/1_5296-1-0.jpg']
# img='../input/test-img/1_5296-1-0.jpg'
# imgs=glob('../input/tensorflow-great-barrier-reef/train_images/video_1/*.jpg')
class_pth = './class.txt'
tmp=np.load('../input/tensorflow-great-barrier-reef/example_test.npy')
inter = Detector(pt_path=pt_path,namesfile=class_pth, img_size=1280,classes=0, conf_thres=0.3)
# img = cv2.imread(img)
# img=img.astype('float64')
for img_tmp in imgs:
    img = cv2.imread(img_tmp)
#     [:, :, ::-1][:, :, ::-1].transpose(2, 0, 1)
    result = inter.detect_image(img)
    bboxes, confs,_=result
#     if len(bboxes)!=0:
#         bboxes=bboxes.astype('int64')
    annot          = format_prediction(bboxes, confs)
    print(result)
print(annot)
# print(tmp.shape)


In [ ]:
print(annot)

In [ ]:
# import numpy as np
# t1=np.load('../input/tensorflow-great-barrier-reef/example_test.npy')[0]
# t2=cv2.imread('../input/tensorflow-great-barrier-reef/train_images/video_1/5296.jpg')
# # [:, :, ::-1]
# # plt.imshow(t, cmap='gray')
# # plt.imshow(t1)
# print(t2)

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission
CONF= 0.15
IOU= 0.50
# im = cv2.imread(img)
# inter = Detector(pt_path=pt_path,namesfile=class_pth, img_size=1280,classes=0)
# result = inter.detect_image(im)
# print(result)
# model = load_model(Best_Model, conf=CONF, iou=IOU)
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
#     im = cv2.imread(img)
#     print(img.shape)
#     bboxes, confs  = predict(model, img, size=IMG_SIZE, augment=True)
#     img=img.astype('float64')
#     img /= 255.0
    img_tmp=img
#     print(img_tmp)
    bboxes, confs,_=inter.detect_image(img_tmp)
#     if len(bboxes)!=0:
#         bboxes=bboxes.astype('int64')
    annot          = format_prediction(bboxes, confs)
    pred_df['annotations'] = annot
    env.predict(pred_df)

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()